## Determining if the 6 Degrees of Separation Theory Works in Selected Boston Marathon Runners

#### DS210 Programming for Data Science Final Project
#### Devin Rauscher (U77872441)

Sources:
* DS210 Lectures 28 and 30 (more so than the others)
* https://en.wikipedia.org/wiki/Six_degrees_of_separation
* https://www.kaggle.com/datasets/zhikchen/boston-marathon-winners-men-and-women
* https://www.kaggle.com/datasets/rojour/boston-results?select=marathon_results_2017.csv
* http://registration.baa.org/2017/cf/Public/iframe_ResultsSearch.cfm
* https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3
* https://notebook.community/konstantinstadler/country_converter/doc/country_converter_examples
* https://doc.rust-lang.org/std/collections/struct.HashSet.html
* https://docs.rs/csv/latest/csv/
* https://docs.rs/crate/bfs/0.1.0
* https://doc.rust-lang.org/book/ch11-01-writing-tests.html

In [12]:
use std::collections::{HashMap, HashSet};
use std::io::{BufRead, BufReader};

fn read_csv(file_path: &str) -> Vec<(String, String, String, u32)> {
    let file = std::fs::File::open(file_path).expect("Failed to open file");
    let reader = BufReader::new(file);
    let mut data = Vec::new();

    for line in reader.lines() {
        if let Ok(row) = line {
            let cols: Vec<&str> = row.split(',').collect();
            if cols.len() == 4 {
                if let (Ok(year), country, name, Ok(win)) = (
                    cols[0].trim().parse::<u32>(),
                    cols[1].trim().to_string(),
                    cols[2].trim().to_string(),
                    cols[3].trim().parse::<u32>(),
                ) {
                    data.push((year.to_string(), country, name, win));
                }
            }
        }
    }

    data
}

fn is_this_tag(text_slice: &str) -> bool {
    text_slice.len() >= 2 && &text_slice[0..=1] == "**"
}

fn build_graph(data: &[(String, String, String, u32)]) -> HashMap<String, HashSet<String>> {
    let mut graph: HashMap<String, HashSet<String>> = HashMap::new();

    for &(ref year, ref country, ref name, _) in data {
        // add the runner's country as a neighbor
        graph.entry(name.to_string()).or_default().insert(country.to_string());

        // find other runners who ran in the same year
        let same_year_runners: Vec<&str> = data
            .iter()
            .filter(|&&(ref y, _, ref n, _)| y == year && name != &n)
            .map(|&(_, _, ref n, _)| n.as_str())
            .collect();

        // add same-year runners as neighbors
        graph
            .entry(name.to_string())
            .or_default()
            .extend(same_year_runners.iter().cloned());

        // add the runner as a neighbor to same-year runners
        for runner in same_year_runners {
            graph
                .entry(runner.to_string())
                .or_default()
                .insert(name.to_string());
        }
    }

    graph
}

// implementing breadth first search
fn bfs(graph: &HashMap<String, HashSet<String>>, start: &str, target: &str) -> Option<usize> {
    let mut queue = vec![(start.to_string(), 0)];
    let mut visited: HashSet<String> = HashSet::new();

    while !queue.is_empty() {
        let (node, level) = queue.remove(0);

        if node == target {
            return Some(level);
        }

        if visited.contains(&node) {
            continue;
        }

        visited.insert(node.clone());

        if let Some(neighbors) = graph.get(&node) {
            for neighbor in neighbors {
                if !visited.contains(neighbor) {
                    queue.push((neighbor.to_string(), level + 1));
                }
            }
        }
    }

    None
}

fn main() {
    // read csv  file
    let file_path = "ds210_dataset.csv";
    let data = read_csv(file_path);

    // build the graph
    let graph = build_graph(&data);

    // test for 6 degrees of separation
    let start_runner = "Runner1";
    let target_runner = "Runner2";

    if let Some(degree) = bfs(&graph, start_runner, target_runner) {
        println!("The degree of separation between {} and {} is {}", start_runner, target_runner, degree);
    } else {
        println!("There is no path between {} and {}", start_runner, target_runner);
    }
}

Error: can't compare `String` with `&String`

Error: expected `Iter<'_, &str>` to be an iterator that yields `&String`, but it yields `&&str`

In [14]:
// testing the usability of the bfs
mod tests {
    use super::*;

    #[test]
    fn test_bfs() {
        // Define a test graph
        let graph: HashMap<String, HashSet<String>> = [
            ("A".to_string(), HashSet::from(["B".to_string(), "C".to_string()])),
            ("B".to_string(), HashSet::from(["A".to_string(), "C".to_string(), "D".to_string()])),
            ("C".to_string(), HashSet::from(["A".to_string(), "B".to_string(), "D".to_string()])),
            ("D".to_string(), HashSet::from(["B".to_string(), "C".to_string(), "E".to_string()])),
            ("E".to_string(), HashSet::from(["D".to_string()])),
        ]
        .iter()
        .cloned()
        .collect();

        // Test case: Find path from "A" to "E"
        let start = "A";
        let target = "E";
        let expected_result = Some(3);
        assert_eq!(bfs(&graph, start, target), expected_result);

        // Test case: Find path from "B" to "C"
        let start = "B";
        let target = "C";
        let expected_result = Some(1);
        assert_eq!(bfs(&graph, start, target), expected_result);

        // Test case: Find path from "A" to "F" (non-existent node)
        let start = "A";
        let target = "F";
        let expected_result = None;
        assert_eq!(bfs(&graph, start, target), expected_result);
    }
}

fn main() {
    // Run the tests
    tests::test_bfs();
}


Error: cannot find function `test_bfs` in module `tests`